In [4]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from ipywidgets import widgets
import plotly.express as px 
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from numpy import fft
import scipy
import pylab as py
from skimage import util

In [5]:
df_pp = pd.read_csv('/home/deaddemocracy/Documents/Major_Project/Extract_Feb18_28_1PM/sensor1/sensor1_pp.csv',decimal = ',',index_col = 0)

In [6]:
df_pp.head()

,ax(g),ay(g),az(g),wx(deg/s),wy(deg/s),wz(deg/s),AngleX(deg),AngleY(deg),AngleZ(deg),T(°),hx,hy,hz
ChipTime,,,,,,,,,,,,,
2020-02-18 15:40:20.960,-0.1851,0.1094,0.9927,-5.7373,-8.0566,5.188,7.3663,11.4148,2.2852,18.11,929,-1405,27
2020-02-18 15:40:21.160,-0.1533,0.0942,1.0044,-6.1646,9.5215,0.2441,6.3062,10.0964,1.7084,18.15,924,-1399,47
2020-02-18 15:40:21.360,-0.1455,0.0859,1.0083,2.4414,1.5259,-1.0986,5.9656,9.5142,1.9391,18.14,896,-1369,51
2020-02-18 15:40:21.560,-0.1299,0.1064,1.0044,3.2959,-5.0659,9.2163,6.7072,8.4705,3.7573,18.14,931,-1369,49
2020-02-18 15:40:21.760,-0.12300000000000001,0.0986,1.0044,-4.5166,-6.5918,2.6245,6.3336,7.5751,4.5593,18.14,1012,-1426,39


# Data Preparation - Feb 23 (12 - 1 PM)


In [7]:
df_pp = df_pp.apply(pd.to_numeric)

In [8]:
df_pp.index = pd.to_datetime(df_pp.index)
df_pp.index = df_pp.index

In [9]:
# plot_data_peak = df_pp['2020-02-19 01:48:30.00':'2020-02-19 01:50:30.00']
plot_data_low = df_pp['2020-02-19 01:48:30.00':'2020-02-19 01:50:30.00']

In [10]:
# plot_ax = pd.to_numeric(plot_data_peak['ax(g)']).round(3)
# plot_ay = pd.to_numeric(plot_data_peak['ay(g)']).round(3)
# plot_az = pd.to_numeric(plot_data_peak['az(g)']).round(3)

plot_ax = pd.to_numeric(plot_data_low['ax(g)']).round(3)
plot_ay = pd.to_numeric(plot_data_low['ay(g)']).round(3)
plot_az = pd.to_numeric(plot_data_low['az(g)']).round(3)

In [11]:
plot_ax_seconds = (plot_ax.index - plot_ax.index[0]).total_seconds()


In [13]:
t = plot_ax_seconds
t

Float64Index([               0.0,                0.2,                0.4,
              0.6000000000000001,                0.8,                1.0,
              1.2000000000000002, 1.4000000000000001,                1.6,
                             1.8,
              ...
              119.00000000000001,              119.2,              119.4,
              119.60000000000001, 119.80000000000001, 120.00000000000001,
                           120.2,              120.4, 120.60000000000001,
              120.80000000000001],
             dtype='float64', name='ChipTime', length=605)

# Sensor Characteristics



In [14]:
#Sampling frequency = 1 / (difference of two consecutive timestamps in seconds)                                                        )

Fs = 1/((df_pp.index[2] - df_pp.index[1]).total_seconds())
Fs

T = 1/Fs

# Data Acquisition Parameters

In [15]:
total_seconds = ((plot_ax.index.max() - plot_ax.index.min()).total_seconds())
total_seconds

120.8

In [16]:
N = len(plot_ax) # Number of samples
N

605

# RMS Plot

In [17]:
def rms_plot(fs,n_samples,time_instant,plot_data):
    w = np.int(np.floor(fs))
    steps = np.int(np.floor(n_samples/w))
    t_RMS = np.zeros((steps,1))
    x_RMS = np.zeros((steps,1))
    
    for i in range(0,steps):
        t_RMS[i] = np.mean(time_instant[(i*w):((i+1)*w)])
        x_RMS[i] = np.sqrt(np.mean(plot_data.values[(i*w):((i+1)*w)]**2))
    
    t_RMS = t_RMS.reshape(t_RMS.shape[0])
    x_RMS = x_RMS.reshape(x_RMS.shape[0])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t_RMS, y=x_RMS, mode='lines',name='lines'))
    
    return t_RMS,x_RMS

## RMS Plot - Feb 23 (12PM-1AM)

In [27]:
# # Plot ax values 
# t_RMS,x_RMS = rms_plot(Fs,N,t,plot_ax)
# fig = go.Figure(data = go.Scatter(x=t_RMS, y=x_RMS, mode='lines',name='lines'))
# fig.update_layout(xaxis_title="Time(s)",yaxis_title="RMS ax(g)")
# fig.show()

# # plot ay values
# t_RMS,y_RMS = rms_plot(Fs,N,t,plot_ay)
# fig2 = go.Figure(data = go.Scatter(x=t_RMS, y=y_RMS, mode='lines',name='lines'))
# fig2.update_layout(xaxis_title="Time(s)",yaxis_title="RMS ay(g)")
# fig2.show()

# # Plot az values
# t_RMS,z_RMS =rms_plot(Fs,N,t,plot_az)
# fig3 = go.Figure(data =go.Scatter(x=t_RMS, y=z_RMS, mode='lines',name='lines'))
# fig3.update_layout(title="Acceleration Plots",xaxis_title="Time(s)",yaxis_title="RMS az(g)")
# fig3.show()



In [28]:
# f, Pxx_den = scipy.signal.welch(plot_az, Fs)
# fig = go.Figure(data = go.Scatter(x=f, y=Pxx_den, mode='lines',name='lines'))
# fig.update_layout(xaxis_title="Frequency(Hz)",yaxis_title="PSD(g^2)", yaxis_type = 'log')
# fig.show()

## RMS Plot - Feb 23 (2AM - 3AM)

In [29]:
# plot_ax_low = pd.to_numeric(plot_data_low['ax(g)']).round(3)
# plot_ay_low = pd.to_numeric(plot_data_low['ay(g)']).round(3)
# plot_az_low = pd.to_numeric(plot_data_low['az(g)']).round(3)

In [30]:
# # Plot ax values 
# t_RMS,x_RMS = rms_plot(Fs,N,t,plot_ax_low)
# fig = go.Figure(data = go.Scatter(x=t_RMS, y=x_RMS, mode='lines',name='lines'))
# fig.update_layout(xaxis_title="Time(s)",yaxis_title="RMS ax(g)")
# fig.show()

# # plot ay values
# t_RMS,y_RMS = rms_plot(Fs,N,t,plot_ay_low)
# fig2 = go.Figure(data = go.Scatter(x=t_RMS, y=y_RMS, mode='lines',name='lines'))
# fig2.update_layout(xaxis_title="Time(s)",yaxis_title="RMS ay(g)")
# fig2.show()

# # Plot az values
# t_RMS,z_RMS =rms_plot(Fs,N,t,plot_az_low)
# fig3 = go.Figure(data =go.Scatter(x=t_RMS, y=z_RMS, mode='lines',name='lines'))
# fig3.update_layout(title="Acceleration Plots",xaxis_title="Time(s)",yaxis_title="RMS az(g)")
# fig3.show()


In [31]:
# f, Pxx_den = scipy.signal.welch(plot_az_low, Fs)
# fig = go.Figure(data = go.Scatter(x=f, y=Pxx_den, mode='lines',name='lines'))
# fig.update_layout(xaxis_title="Frequency(Hz)",yaxis_title="PSD(g^2)", yaxis_type = 'log')
# fig.show()

In [23]:
def fft_plot(T_period,N_samples,array_in):
    xf = np.linspace(0.0, 1.0/(2.0*T), N/2);
    yf = np.fft.fft(array_in);
    plot_xaxis = xf
    plot_yaxis = (2.0/N) * np.abs(yf[0:np.int(N/2)])
    return plot_xaxis,plot_yaxis

## FFT Plot - Feb 23(12PM - 1PM)

In [25]:
# # FFT Plot of ax
# plot_xaxis_x,plot_yaxis_x = fft_plot(T,N,plot_ax)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=plot_xaxis_x, y=plot_yaxis_x, mode='lines',name='lines'))
# fig.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(x)",xaxis_type='log',yaxis_type='log')
# fig.show()

# # FFT Plot of ay
# plot_xaxis_y,plot_yaxis_y = fft_plot(T,N,plot_ay)
# fig2 = go.Figure()
# fig2.add_trace(go.Scatter(x=plot_xaxis_y, y=plot_yaxis_y, mode='lines',name='lines'))
# fig2.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(y)",xaxis_type='log',yaxis_type='log')
# fig.show()

# # FFT Plot of ay
# plot_xaxis_z,plot_yaxis_z = fft_plot(T,N,plot_az)
# fig3 = go.Figure()
# fig3.add_trace(go.Scatter(x=plot_xaxis_z, y=plot_yaxis_z, mode='lines',name='lines'))
# fig3.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(z)",xaxis_type='log',yaxis_type='log')
# fig3.show()



## FFT Plot - Feb 23 (2AM - 3AM)


In [26]:
# # FFT Plot of ax
# plot_xaxis_x_low,plot_yaxis_x_low = fft_plot(T,N,w,plot_ax_low)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=plot_xaxis_x_low, y=plot_yaxis_x_low, mode='markers',name='lines'))
# fig.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(x)",xaxis_type='log',yaxis_type='log')
# fig.show()

# # FFT Plot of ay
# plot_xaxis_y_low,plot_yaxis_y_low = fft_plot(T,N,w,plot_ay_low)
# fig2 = go.Figure()
# fig2.add_trace(go.Scatter(x=plot_xaxis_y_low, y=plot_yaxis_y_low, mode='lines',name='lines'))
# fig2.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(y)",xaxis_type='log',yaxis_type='log')
# fig.show()

# # FFT Plot of ay
# plot_xaxis_z_low,plot_yaxis_z_low = fft_plot(T,N,w,plot_az_low)
# fig3 = go.Figure()
# fig3.add_trace(go.Scatter(x=plot_xaxis_z_low, y=plot_yaxis_z_low, mode='lines',name='lines'))
# fig3.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(z)",xaxis_type='log',yaxis_type='log')
# fig3.show()



## az - RMS Plot (Feb 19 1:48:30 AM to 1:50:30 AM)

In [34]:
plot_az

ChipTime
2020-02-19 01:48:30.160    1.016
2020-02-19 01:48:30.360    1.016
2020-02-19 01:48:30.560    1.016
2020-02-19 01:48:30.760    1.016
2020-02-19 01:48:30.960    1.016
                           ...  
2020-02-19 01:50:30.160    1.016
2020-02-19 01:50:30.360    1.016
2020-02-19 01:50:30.560    1.016
2020-02-19 01:50:30.760    1.016
2020-02-19 01:50:30.960    1.016
Name: az(g), Length: 605, dtype: float64

In [38]:
# FFT Plot of ay
plot_xaxis,plot_yaxis = fft_plot(T,N,plot_az)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=plot_xaxis, y=plot_yaxis, mode='lines',name='lines'))
fig3.update_layout(title="FFT",xaxis_title="Frequency",yaxis_title ="amplitude(z)",yaxis_type='log')
fig3.show()
